# Connexion au cluster

La connexion est possible uniquement depuis les PCs des salles de TP  

```bash
$ ssh pnavaro@cluster-infomaths.insa-rennes.fr
The authenticity of host 'cluster-infomaths.insa-rennes.fr (193.52.94.112)' can't be established.
ECDSA key fingerprint is SHA256:wEtf2Oc8PKFls/D9oDQF4vXcs9y2ku0i+At4KiVJ04Q.
Are you sure you want to continue connecting (yes/no)? yes
Warning: Permanently added 'cluster-infomaths.insa-rennes.fr,193.52.94.112' (ECDSA) to the list of known hosts.
pnavaro@cluster-infomaths.insa-rennes.fr's password:
Linux cim-tete 4.9.0-4-amd64 #1 SMP Debian 4.9.65-3+deb9u1 (2017-12-23) x86_64

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
Last login: Tue Oct 16 09:02:56 2018 from 10.4.1.36
pnavaro@cim-tete:~$ 
```

# Activer l'environnement conda big-data

```bash
source /opt/miniconda3/etc/profile.d/conda.sh
conda activate big-data
```

# Commandes de base

- **sbatch**  : soumission d'un job dans une partition.
- **scancel** : suppression d'un job.
- **squeue**  : interrogation des jobs.
- **sinfo**   : interrogation des files d'attente.
- **srun**    : execution immediate d'une commande.
- **salloc**  : batch interactif, obtention d'un shell, permettant d'enchaîner plusieurs commandes sur les mêmes ressources.  
- **sprio**   : priorités relatives entre les jobs en attente  

Une documentation des commandes de base est disponible ici : [http://slurm.schedmd.com/man_index.html](http://slurm.schedmd.com/man_index.html) 

# Utilisation du Cluster

## Ressources disponibles

- **Noeuds :** 4 (cim-n01 / cim-n02 / cim-n03 / cim-n04)  
- **CPU par noeud :** 24  
- **Socket par noeud :** 1  
- **Coeur par socket :** 12  
- **Threads par coeur :** 2  
- **RAM par noeud :** 64Go  
- **Nom du cluster :** cluster-infomath  
- **Nom de la partition :** queue 

## Lancement d'un job

Pour lancer un job, il faut créer un script dans lequel il faut demander des ressources puis appeler son programme.  

Ce script est ensuite soumi au gestionnaire de file d'attente avec la commande **sbatch**. Par exemple :  

```
$ sbatch mon_script.sh
```

On obtient alors un numéro de job, qui peut être manipulé avec les commandes `scancel` ou `squeue`.  

Le script peut être écrit dans le langage de son choix (bash, tcsh, python, perl...). Il peut être exécuté directement, sans être appelé par sbatch, et dans ce cas, les directives d'allocations de ressources seront ignorées, et il s'exécutera dans le shell local.  

`
## Demande de ressources

Vos besoins en terme de ressources sont décrits dans l'en-tête d'un fichier via des directives Slurm. Par exemple :  

```slurm
#SBATCH -N 4  
#SBATCH --tasks-per-node 5
```

demandera une allocation de 4 noeuds à 5 coeurs chacun (par défaut, 1 tâche demande 1 coeur).

D'autres critères peuvent être spécifié via ces directives, comme la taille mémoire souhaitée ou la durée pendant laquelle les ressources seront attribuées.  

**Plus vous serez parcimonieux dans la demande d'allocation, plus vous aurez de chance de voir rapidement votre job passer de l'état en attente à l'état en exécution.**  

Par exemple, s'il est possible d'estimer précisément la durée nécessaire à une exécution, il peut être profitable de réduire au minimum la durée demandée pour la réservation. Ainsi une exécution se faisant en 3h30 pourra se faire au sein d'un job demandant 4h00 (marge de 30mn par précaution), avec la directive  

```
#SBATCH -t 04:00:00  
```

Attention toutefois à prendre une marge suffisante, car au-delà du temps demandé, l'exécution est stoppée automatiquement par Slurm.

Attention : toutes les partitions sont configurées avec une limite de temps d'exécution par défaut, qui s'applique à tout job ne précisant pas combien de temps doit lui être alloué. Pour connaître cette limite, utiliser la commande suivante :  

```
sinfo -o "%10P %.11L %.11l"
```

Remarque : tous les arguments de la directive #SBATCH peuvent également être utilisés en arguments des commandes srun, salloc et sbatch. Voir les exemples plus loin.

## Suivre l'état d'un job

Il est possible d'obtenir le détail de l'état d'un job, qu'il soit en attente ou en exécution, avec les commandes scontrol ou squeue. La commande sview permet également en partie d'obtenir ces informations.  
**Liste des jobs en cours :**  

```
squeue -u $USER
```

**Mon job est en attente. Quand va-t-il démarrer ?**  

Pour les jobs en attente, Slurm calcul périodiquement un temps probable de démarrage.  

2 possibilités pour l'obtenir:  

```
scontrol show job <jobid>| grep StartTime=</jobid>
```

```
squeue -o "%S" -j
```

## Variables d'environnement

Dans certains cas, votre programme a besoin de connaître plus précisément les ressources que Slurm a mis à sa disposition. Pour cela, Slurm fourni un certain nombre de variables d'environnement, qui sont utilisable dans le script qui appelle le programme.  

- SLURM_NPROCS : nombre de coeurs alloués  
- SLURM_NNODES : nombre de noeuds alloués  
- SLURM_JOB_ID : job id  
- SLURM_JOB_NODELIST : liste des noeuds alloués, sous une forme synthétique. Pour obtenir une liste détaillée, on peut utiliser la commande "scontrol show hostname"

## Priorités

Quand plusieurs jobs sont en même temps en attente dans une file, Slurm calcule une priorité entre ces jobs. Le job ayant la priorité la plus élevée sera le prochain à passer en exécution.  

La priorité des jobs peut être vue avec la commande `sprio -l`.  

La priorité dépend de plusieurs facteurs :  
- le temps d'attente déjà écoulé (AGE)  
- la taille du job en nombre de coeurs : les gros jobs sont favorisés (JOBSIZE)  
- la consommation en heures cpu sur le passé récent : plus la consommation a été faible, plus la priorité augmentera (FAIRSHARE).  

## Exemple de script Slurm

Ce script demande 16 coeurs répartis sur au minimum sur 1 noeuds et au maximum sur 2 noeuds :  

```bash
#! /bin/bash  
#SBATCH -n 16 # 12 coeurs  
#SBATCH -N 1-2  # au minimum 3 noeuds, au maximum 6  
#SBATCH -t 00:10:00 # Le job sera tué au bout de 10 minutes  
#SBATCH --mem=1024 # Quantité mémoire demandée par noeud en Mo (unité obligatoire)  
#SBATCH --mail-type=END # Réception d'un mail à la fin du job  
#SBATCH --mail-user=mon@adresse  

source /opt/miniconda3/etc/profile.d/conda.sh
conda activate big-data
python keras_example.py
```